<a href="https://colab.research.google.com/github/Aboelsaood23/Data-Sets/blob/main/Rental_Properties_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#connect colab to drie
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

file_path = "/content/drive/MyDrive/Colab Notebooks/full_Data_Projects/property_rentals.csv"
df =pd.read_csv(file_path)
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,id,latitude,longitude,property_type,room_type,bathrooms,bedrooms,minimum_nights,price
0,958,37.76931,-122.43386,Apartment,Entire home/apt,1.0,1.0,1,$170.00
1,3850,37.75402,-122.45805,House,Private room,1.0,1.0,1,$99.00
2,5858,37.74511,-122.42102,Apartment,Entire home/apt,1.0,2.0,30,$235.00
3,7918,37.76669,-122.45250,Apartment,Private room,4.0,1.0,32,$65.00
4,8142,37.76487,-122.45183,Apartment,Private room,4.0,1.0,32,$65.00


In [ ]:
print(df.dtypes)

id                  int64
latitude          float64
longitude         float64
property_type      object
room_type          object
bathrooms         float64
bedrooms          float64
minimum_nights      int64
price              object
dtype: object


In [ ]:
#remove colume of longitude , latitude,bathrooms, minmum_nights and room_type
df =df.drop(['longitude','latitude','bathrooms','room_type','minimum_nights'],axis=1)
# remove dollar sign from price and convert it to float
df['price'] = df['price'].str.replace('$', '', regex=False)     # Remove $
df['price'] = df['price'].str.replace(',', '', regex=False)     # Remove commas
df['price'] = df['price'].str.replace('"', '', regex=False)     # Remove double quotes
df['price'] = df['price'].str.strip()                           # Remove leading/trailing spaces

df['price'] = df['price'].astype(float)                         # Convert to float


df =df.rename(columns={'price':'price_$'})
print(df.dtypes)
df.head()

id                 int64
property_type     object
bedrooms         float64
price_$          float64
dtype: object


,id,property_type,bedrooms,price_$
0,958,Apartment,1.0,170.0
1,3850,House,1.0,99.0
2,5858,Apartment,2.0,235.0
3,7918,Apartment,1.0,65.0
4,8142,Apartment,1.0,65.0


In [ ]:
import numpy as np
#add new column
df['city']= "New York"
df.head()
# add neighborhood column
neighborhoods =['Manhattan','Brookyn','Queens','Harlem','Staten Island']
df['neighborhood'] =np.resize(neighborhoods,df.shape[0]) #repaet neighborhoods to match number of rows
# we need to shuffle it to reduce redundancy
df = df.sample(frac=1).reset_index(drop=True) #shuffle and reset index
df.head()

,id,property_type,bedrooms,price_$,city,neighborhood
0,9021715,House,2.0,299.0,New York,Brookyn
1,23630,House,1.0,155.0,New York,Harlem
2,9949873,House,1.0,90.0,New York,Staten Island
3,4833101,House,1.0,120.0,New York,Manhattan
4,216042,House,1.0,147.0,New York,Harlem


In [ ]:
#let's add two more columns [start date,end date]
from datetime import datetime,timedelta

n =len(df)
start_min =datetime(2024,4,1)
start_max =datetime(2024,12,30)
end_max = datetime(2024, 12, 31)
start_dates = [start_min + timedelta(days=np.random.randint(0, (start_max - start_min).days + 1)) for _ in range(n)]
end_dates = [
    start + timedelta(days=np.random.randint(1, (end_max - start).days + 1))
    for start in start_dates
]
df['start_date']=start_dates
df['end_date']=end_dates
df.head()


,id,property_type,bedrooms,price_$,city,neighborhood,start_date,end_date
0,9021715,House,2.0,299.0,New York,Brookyn,2024-08-16,2024-08-22
1,23630,House,1.0,155.0,New York,Harlem,2024-04-15,2024-11-02
2,9949873,House,1.0,90.0,New York,Staten Island,2024-07-22,2024-09-15
3,4833101,House,1.0,120.0,New York,Manhattan,2024-05-17,2024-10-12
4,216042,House,1.0,147.0,New York,Harlem,2024-08-22,2024-09-20


In [ ]:
# we need to replace any other property type but house apartment with name : studio
names =df['property_type'].unique()
print(names)
df['property_type'] = df['property_type'].where(
    df['property_type'].isin(['Apartment', 'House']),
    other='Studio'
)
df.head()

['House' 'Loft' 'Apartment' 'Hotel' 'Townhouse' 'Serviced apartment'
 'Condominium' 'Guest suite' 'Guesthouse' 'Boutique hotel' 'Hostel'
 'Bed and breakfast' 'Cottage' 'Bungalow' 'Other']


,id,property_type,bedrooms,price_$,city,neighborhood,start_date,end_date
0,9021715,House,2.0,299.0,New York,Brookyn,2024-08-16,2024-08-22
1,23630,House,1.0,155.0,New York,Harlem,2024-04-15,2024-11-02
2,9949873,House,1.0,90.0,New York,Staten Island,2024-07-22,2024-09-15
3,4833101,House,1.0,120.0,New York,Manhattan,2024-05-17,2024-10-12
4,216042,House,1.0,147.0,New York,Harlem,2024-08-22,2024-09-20


In [ ]:
names =df['property_type'].unique()
print(names)

['House' 'Studio' 'Apartment']


In [ ]:
# add new column for parking fess
def assign_parking_fees(property_type):
  if property_type =="Studio":
    return round(np.random.uniform(15,30),2)
  elif property_type =="Apartment":
    return round(np.random.uniform(25,40),2)
  elif property_type=="House":
    return 0.00
  else:
    return np.nan

df['parking_fee_per_month_$'] =df['property_type'].apply(assign_parking_fees)


# add another column for laundry
def assign_laundry_fee(property_type):
  if property_type=="Studio":
    return round(np.random.uniform(20,30),2)
  elif property_type == "Apartment":
    return round(np.random.uniform(25,40),2)
  elif property_type =="House":
    return round(np.random.uniform(35,50),2)
  else :
    return np.nan

df['laundry_per_month_$']= df['property_type'].apply(assign_laundry_fee)


# add new column for claening times per month 4 for studio , 3 for appartment and 2 for hous
def assign_cleaning_times_per_month (property_type):
  if property_type=="Studio":
    return 4
  elif property_type == "Apartment":
    return 3
  elif property_type =="House":
    return 2
  else :
    return np.nan

df['cleaning_times_per_month']= df['property_type'].apply(assign_cleaning_times_per_month)

#add new column for room cleaing price
def room_cleaning_price (property_type):
  if property_type=="Studio":
    return 5
  elif property_type == "Apartment":
    return 7
  elif property_type =="House":
    return 8
  else :
    return np.nan

df['room_cleaining_price_$']= df['property_type'].apply(room_cleaning_price)


# new column for mantainance

def assign_mantainance_fee(property_type):
  if property_type=="Studio":
    return round(np.random.uniform(35,50),2)
  elif property_type == "Apartment":
    return round(np.random.uniform(45,70),2)
  elif property_type =="House":
    return round(np.random.uniform(60,80),2)
  else :
    return np.nan

df['mantainance_per_month_$']= df['property_type'].apply(assign_mantainance_fee)


# add tax column to dataset
df['tax$_$'] =df['price_$']*0.04
df.head()

,id,property_type,bedrooms,price_$,city,neighborhood,start_date,end_date,parking_fee_per_month_$,laundry_per_month_$,cleaning_times_per_month,room_cleaining_price_$,mantainance_per_month_$,tax$_$
0,9021715,House,2.0,299.0,New York,Brookyn,2024-08-16,2024-08-22,0.0,41.59,2,8,71.53,11.96
1,23630,House,1.0,155.0,New York,Harlem,2024-04-15,2024-11-02,0.0,49.49,2,8,61.37,6.20
2,9949873,House,1.0,90.0,New York,Staten Island,2024-07-22,2024-09-15,0.0,46.85,2,8,78.01,3.60
3,4833101,House,1.0,120.0,New York,Manhattan,2024-05-17,2024-10-12,0.0,44.02,2,8,70.23,4.80
4,216042,House,1.0,147.0,New York,Harlem,2024-08-22,2024-09-20,0.0,49.60,2,8,67.63,5.88


In [ ]:
df.head()

,id,property_type,bedrooms,price_$,city,neighborhood,start_date,end_date,parking_fee_per_month_$,laundry_per_month_$,cleaning_times_per_month,room_cleaining_price_$,mantainance_per_month_$,tax$_$
0,9021715,House,2.0,299.0,New York,Brookyn,2024-08-16,2024-08-22,0.0,41.59,2,8,71.53,11.96
1,23630,House,1.0,155.0,New York,Harlem,2024-04-15,2024-11-02,0.0,49.49,2,8,61.37,6.20
2,9949873,House,1.0,90.0,New York,Staten Island,2024-07-22,2024-09-15,0.0,46.85,2,8,78.01,3.60
3,4833101,House,1.0,120.0,New York,Manhattan,2024-05-17,2024-10-12,0.0,44.02,2,8,70.23,4.80
4,216042,House,1.0,147.0,New York,Harlem,2024-08-22,2024-09-20,0.0,49.60,2,8,67.63,5.88


In [ ]:
# data expansion
from pandas.tseries.offsets import MonthEnd,MonthBegin
df['start_date'] = pd.to_datetime(df['start_date'])
df['end_date'] = pd.to_datetime(df['end_date'])

expanded_rows = []

for _, row in df.iterrows():
    current = row['start_date'].replace(day=1)
    end_month = row['end_date'].replace(day=1)

    while current <= end_month:
        month_start = current
        month_end = current + MonthEnd(0)

        # Calculate actual period for this month
        actual_start = max(row['start_date'], month_start)
        actual_end = min(row['end_date'], month_end)

        days_in_month = (actual_end - actual_start).days + 1

        # Build the expanded row
        expanded_row = row.to_dict()
        expanded_row['year'] = current.year
        expanded_row['month'] = current.strftime('%b')
        expanded_row['days_in_month'] = days_in_month

        expanded_rows.append(expanded_row)

        # Move to next month
        current += MonthBegin(1)

# Final expanded DataFrame
expanded_df = pd.DataFrame(expanded_rows)

# Reorder to keep original columns first, then add new onesoriginal_cols = list(df.columns)
original_cols = list(df.columns)
new_cols = ['year', 'month', 'days_in_month']
expanded_df = expanded_df[original_cols + new_cols]

# Output
expanded_df.head()


,id,property_type,bedrooms,price_$,city,neighborhood,start_date,end_date,parking_fee_per_month_$,laundry_per_month_$,cleaning_times_per_month,room_cleaining_price_$,mantainance_per_month_$,tax$_$,year,month,days_in_month
0,9021715,House,2.0,299.0,New York,Brookyn,2024-08-16,2024-08-22,0.0,41.59,2,8,71.53,11.96,2024,Aug,7
1,23630,House,1.0,155.0,New York,Harlem,2024-04-15,2024-11-02,0.0,49.49,2,8,61.37,6.20,2024,Apr,16
2,23630,House,1.0,155.0,New York,Harlem,2024-04-15,2024-11-02,0.0,49.49,2,8,61.37,6.20,2024,May,31
3,23630,House,1.0,155.0,New York,Harlem,2024-04-15,2024-11-02,0.0,49.49,2,8,61.37,6.20,2024,Jun,30
4,23630,House,1.0,155.0,New York,Harlem,2024-04-15,2024-11-02,0.0,49.49,2,8,61.37,6.20,2024,Jul,31


In [ ]:
#download this new datset
expanded_df.to_csv('Rental_properties_data.csv',index=False)

from google.colab import files
files.download('Rental_properties_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>